# Cropping Object frames
* The code written is based on methods from [LeagueAI](https://arxiv.org/abs/1905.13546) and the associated [Github repo](https://github.com/Oleffa/LeagueAI)
## Methodology
1. Removing the unicolor background of the images by modifying the alpha channel(for transparency) of all unicolor pixels.
2. Image is cropped.
3. Cropped dimensions will serve as bounding boxes.

In [1]:
import os
from PIL import Image, ImageFilter
import numpy as np
from os import listdir
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm

In [2]:
def get_files_in_dir_as_dict(path, filt='.csv'):
    ''' Get all files from path. Returns a dict of folder + path'''
    assert os.path.exists(path), "The path {} was not found!".format(path)
    f = dict()
    for (dirpath, dirnames, filenames) in os.walk(path):
        files = [os.path.join(dirpath,f) for f in filenames if filt in f]
        if len(files) > 0:
            f[dirpath] = files
    return f
def get_files_in_dir_as_list(path, filt='.csv'):
    f_dict = get_files_in_dir_as_dict(path, filt)
    return [k for key in f_dict for k in f_dict[key]]

In [3]:
# Area to pre-crop the images to (min_x, min_y, max_x, max_y), can save runtime for large screenshots with small objects
area = (700,300,1240,780)

tolerance_offset_1 = 1.0
tolerance_offset_2 = 1.0 # Greenscreen: 0.74
tolerance_offset_3 = 2.5 # Teemo viewer: 2.5
tolerance1 = tolerance_offset_1*25
tolerance2 = tolerance_offset_2*25
tolerance3 = tolerance_offset_3*25
tolerance = (tolerance1,tolerance2,tolerance3)
background = (0,255,0)

In [4]:
def rgba_channel_equal(c1, c2, tolerance):
    return ((abs(int(c1[0]) - int(c2[0])) <= tolerance[0]) and 
            (abs(int(c1[1]) - int(c2[1])) <= tolerance[1]) and 
            (abs(int(c1[2]) - int(c2[2])) <= tolerance[2]))

def modify_outline(image, thickness):
    #image = Image.open(f)
    #image = image.convert("RGBA")
    
    for t in range(thickness):
        mask = image.filter(ImageFilter.FIND_EDGES)
        mask_data = mask.getdata()
        image_data = image.getdata()
        w, h = mask_data.size
        
        out_data=[]
        for y in range(0, h):
            for x in range(0, w):
                index = x + w*y
                pixel = (0,0,0,0)
                if mask_data[index][3]>0:
                    pixel = (255,255,255, 0)
                else:
                    pixel = (image_data[index][0], image_data[index][1],  image_data[index][2],  image_data[index][3])
                out_data.append(pixel)
        image.putdata(out_data)
    #image.save(out)
    return image

def get_y_min_max(new_data, w, h, scan_step=5, channel=1, channel_mask=(0,255,0), tolerance=(10,10,10)):
    '''Get min max values for y axis. Axis = x'''
    min_value = 0
    max_value = 0
    for y in range(h-1, 0,-scan_step):
        for x in range(0, w-1):
            data_index = x + w * y
            #if abs(new_data[data_index][1] - 255) > 40:
            if not rgba_channel_equal(new_data[data_index], channel_mask, tolerance):
                #print('max = {}'.format(new_data[data_index][1]))
                max_value = y
                break
        else:
            continue
        break
        
    for y in range(0, h-1, scan_step):
        for x in range(0, w-1):
            data_index = x + w * y
            #if abs(new_data[data_index][1] - 255) > 40: 
            if not rgba_channel_equal(new_data[data_index], channel_mask, tolerance):
                min_value = y
                #print('min = {}'.format(new_data[data_index][1]))
                break
        else:
            continue
        break
    return min_value-scan_step, max_value+scan_step

def get_xy_min_max(tgt_img, scan_step=1, channel=1, channel_mask=(0,255,0), tolerance=(10,10,10)):
    # y axis
    img_data = tgt_img.getdata()
    w,h = tgt_img.size
    min_y, max_y = get_y_min_max(img_data,w=w,h=h, 
                                 channel=channel, 
                                 channel_mask=channel_mask, 
                                 tolerance=tolerance,
                                scan_step=scan_step)
    
    # X axis
    img = tgt_img.rotate(-90,expand=True)
    img_data = img.getdata()
    w,h = img.size
    min_x,max_x = get_y_min_max(img_data,w=w,h=h, 
                                 channel=channel, 
                                 channel_mask=channel_mask, 
                                 tolerance=tolerance,
                                scan_step=scan_step)
    
    return min_x, min_y, max_x, max_y-scan_step//2

In [36]:
def img_2_cropped_png(input_img_path, background=(0,255,0), tolerance=(10,40,10) ):
    tolerance1, tolerance2, tolerance3 = tolerance
    img = Image.open(input_img_path)
    # Add alpha channel
    img = img.convert("RGBA")
    img = img.crop((100,10,800,400))
    dim = get_xy_min_max(img, channel_mask=background, tolerance=(5,40,5), scan_step=10)
    img_cropped = img.crop((dim))
    datas = img_cropped.getdata()
    newData = list(datas)
    
    for idx, item in zip(range(len(datas)), datas):
        if (abs(item[0] - background[0]) < tolerance1 and 
         abs(item[1] - background[1]) < tolerance2 and 
         abs(item[2] - background[2]) < tolerance3): 
             newData[idx] = (255,255,255,0)
        else:
            newData[idx] = (item[0], item[1], item[2], 255)
            
    img_cropped.putdata(newData)
    w,h = img_cropped.size
    # Crop image to pixel content
    dim = get_xy_min_max(img_cropped, channel_mask=background, tolerance=(5,40,5), scan_step=1)
    # Save output image as png
    img_cropped = img_cropped.crop(dim)
    img_cropped = modify_outline(img_cropped, 1)
    #img_cropped.save()
    return img_cropped

def img_2_cropped_png_wrapper(input_img_path, background=(0,255,0),out_path= 'D:/SK MSc/II/LoL_Object_Detection/creeps_cropped', root_dir = 'D:/SK MSc/II/LoL_Object_Detection/creeps'):
    img = img_2_cropped_png(input_img_path, background=background, tolerance=tolerance)
    rel_path = os.path.relpath(input_img_path, root_dir)
    save_path = os.path.join(out_path, os.path.dirname(rel_path))
    #print(save_path)
    base_name = os.path.basename(input_img_path)
    if(not os.path.exists(save_path)):
        os.makedirs(save_path)
    #print(save_path)
    img.save(os.path.join(save_path,base_name))
    return True

# Crop Champion Images

In [6]:
root_dir = 'champions'
image_paths = get_files_in_dir_as_list('champions/', 'png')
out_path = 'D:/SK MSc/II/LoL_Object_Detection/chapions_cropped2' ########################TODO!!

In [ ]:
 rel_path = os.path.relpath(input_img_path, root_dir)

In [7]:
len(image_paths)

10472

In [8]:
for img in tqdm(image_paths):
    img_2_cropped_png_wrapper(img,)

100%|████████████████████████████████████████████████████████████████████████████| 10472/10472 [31:44<00:00,  5.50it/s]


# Crop creep images

In [19]:
background = (4, 244, 4) # Green screen
out_path = 'D:/SK MSc/II/LoL_Object_Detection/creeps_cropped/'

In [20]:
out_path

'D:/SK MSc/II/LoL_Object_Detection/creeps_cropped/'

In [39]:
creep_image_paths = get_files_in_dir_as_list('D:/SK MSc/II/LoL_Object_Detection/creeps', 'png')

In [43]:
for img in tqdm(creep_image_paths):
    img_2_cropped_png_wrapper(input_img_path=img, background=background, out_path=out_path,root_dir='D:/SK MSc/II/LoL_Object_Detection/creeps')

100%|██████████████████████████████████████████████████████████████████████████████| 9980/9980 [25:46<00:00,  6.45it/s]
